# Inisiasi Kebutuhan library dan functions

In [ ]:
# Import library

# library kebutuhan
import os
import datetime
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# library algoritma
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

# Macam macam algoritma

# komen paket yang tidak digunakan
algorithm_pack = [[GaussianNB(), 'Naive Bayes', 'NB'],
                [KNeighborsClassifier(), 'K Nearest Neighbors', 'KNN'],
                [SVC(), 'Support Vector Machine','SVM'],
                [GradientBoostingClassifier(random_state=100,), 'Gradient Boosting', 'GB'],
                [DecisionTreeClassifier(random_state=100), 'Decision Tree', 'DT'],
                [RandomForestClassifier(random_state=100), 'Random Forest','RF'],
                [MLPClassifier(max_iter=1000), 'Multi-layer Perceptron', 'MLP']]

# Fungsi untuk menampilkan opsi algoritma apa yang ingin dipakai
def algorithm_option_string():
    string = ''
    for i in range(len(algorithm_pack)):
        string += f"{i}. {algorithm_pack[i][1]}"
    string += '\n['
    for i in range(len(algorithm_pack)):
        if(i == len(algorithm_pack) - 1):
            string+= f"{i}]: "
        else:
            string+= f"{i}/"
    return string

# Fungsi untuk print error dengan warna merah dan diberi jarak
def print_error(string):
    print("\n\033[91m"+string+"\033[0m\n")

# Fungsi untuk validasi input opsi integer
def get_integer_option(min_value, max_value, prompt):
    while True:
        try:
            value = int(input(prompt))
            if min_value <= value <= max_value:
                return value
            else:
                print_error(f"Option must be between {min_value} and {max_value}")
        except ValueError:
            print_error(f"Invalid option. Please enter a valid integer value between {min_value} and {max_value}")

def get_current_time(format=None):
    """Gets the current time in Python and optionally formats it.

    Args:
        format (str, optional): The desired format for the time string.
            Defaults to None, which returns the time in ISO 8601 format (YYYY-MM-DD HH:MM:SS).
            Common format codes include:
                - %Y: Year (e.g., 2024)
                - %m: Month (e.g., 02)
                - %d: Day of the month (e.g., 27)
                - %H: Hour in 24-hour format (e.g., 16)
                - %M: Minute (e.g., 58)
                - %S: Second (e.g., 24)
                - %f: Microseconds (e.g., 406569)

    Returns:
        str: The formatted current time string.
    """

    current_time = datetime.datetime.now()

    if format:
        return current_time.strftime(format)
    else:
        return current_time.isoformat()
    
current_time = get_current_time("%d-%m-%y_%H=%M=%S")
    
def remove_file(filename):
    if os.path.exists(filename):
        os.remove(filename)

# -- Baca data dan menyimpan data original

In [ ]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
origin_test_data = test_data

# Melihat Info dari train data

In [ ]:
train_data.head()

In [ ]:
train_data.info()

In [ ]:
train_data.describe()

# -- Drop kolom yang tidak relevan dan set kolom yang ingin diprediksi

In [ ]:
# Ganti isi dengan fitur yang sesuai dengan data
dropped_columns = ['id', 'hospital_number', 'lesion_2', 'lesion_3']
target_column = 'outcome'

categorical_features = train_data.drop(columns=dropped_columns + [target_column]).select_dtypes(include=['object']).columns.tolist()
numerical_features = train_data.drop(columns=dropped_columns + [target_column]).select_dtypes(exclude=['object']).columns.tolist()

train_data = train_data.drop(columns=dropped_columns)
test_data = test_data.drop(columns=dropped_columns)

# Melihat kembali apakah fitur kategorikal dan fitur numerikal sudah benar

In [ ]:
print(categorical_features)
print(numerical_features)

# Melakukan Preprocessing sesuai kebutuhan pada data sebelum di latih

In [ ]:
def null_to_none(s):
    if isinstance(s, str):
        return s
    else:
        return "none"

In [ ]:
for categories in categorical_features:
    train_data[categories] =train_data[categories].apply(null_to_none)
    test_data[categories] =test_data[categories].apply(null_to_none)

In [ ]:
train_data.T

In [ ]:
test_data.T

In [ ]:
ordinal_categories = ['temp_of_extremities', 'peripheral_pulse', 'capillary_refill_time', 'pain', 'abdominal_distention', 'nasogastric_tube', 'nasogastric_reflux', 'rectal_exam_feces', 'abdomen']
non_ordinal_categories = [element for element in categorical_features if element not in ordinal_categories]

In [ ]:
print(non_ordinal_categories)

In [ ]:
for cols in ordinal_categories:
    print(cols + ": ")
    print(train_data[cols].unique())

In [ ]:
ordinal_orders = [
    ['none', 'warm', 'normal', 'cool', 'cold'], # Kolom temp_of_extremities
    ['none', 'absent', 'reduced', 'normal', 'increased'], # Kolom peripheral_pulse
    ['none', 'less_3_sec', '3', 'more_3_sec'], # Kolom capillary_refill_time
    ['none', 'slight', 'mild_pain', 'alert', 'depressed', 'severe_pain', 'extreme_pain'], # Kolom pain
    ['none', 'slight', 'moderate', 'severe'], # Kolom abdominal_distention
    ['none', 'slight', 'significant'], # Kolom nasogastric_tube
    ['none', 'slight', 'less_1_liter', 'more_1_liter'], # Kolom nasogastric_reflux
    ['none', 'absent', 'decreased', 'normal', 'increased', 'serosanguious'], # Kolom rectal_exam_feces
    ['none', 'normal', 'firm', 'distend_small', 'distend_large', 'other'] # Kolom abdomen
]

In [ ]:
# Ini adalah fungsi transformasi yang digunakan untuk fitur kategorikal non ordinal
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(sparse_output=False))
])  # OneHotEncoder mengubah kategorikal menjadi biner (numerikal)

# Ini adalah fungsi transformasi yang digunakan untuk fitur kategorikal ordinal
ordinal_transformer = Pipeline(steps=[
    ('ordinal', OrdinalEncoder(categories=ordinal_orders))
])

# Ini adalah fungsi transformasi yang digunakan untuk fitur numerikal non ordinal
numerical_imputer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),   # Mengisi null value dengan data median
    ('scaler', StandardScaler())                    # Melakukan standarisasi skalar
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, non_ordinal_categories),
        ('ord', ordinal_transformer, ordinal_categories),
        ('num', numerical_imputer, numerical_features)
    ], remainder='passthrough')

pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# -- Mulai memisah menjadi x_train, y_train, dan x_test (tidak perlu y_test karena yang diminta hasil prediksi bukan akurasi)

In [ ]:
x_train_visualize = train_data
x_train = train_data.drop(columns=[target_column])
y_train = train_data[target_column]
x_test = test_data

In [ ]:
x_train.T

In [ ]:
x_test.T

In [ ]:
x_train_visualize = pipeline.fit_transform(x_train_visualize)
x_train = pipeline.fit_transform(x_train)
x_test = pipeline.transform(x_test)

all_column_names = preprocessor.get_feature_names_out()
all_column_names = [name[name.find("__") + 2:] for name in all_column_names]

# -- Fungsi untuk output csv, diganti berdasarkan kebutuhan

## Fungsi untuk prediksi bersifat non probabilitas

In [ ]:
def output_one_csv(i):
    # Algoritma untuk output prediksi kedalam csv
    algorithm_pack[i][0].fit(x_train, y_train)
    predictions = algorithm_pack[i][0].predict(x_test)
    ids = origin_test_data['id']
    output_df = pd.DataFrame({'id': ids, 'outcome': predictions})
    output_csv_name = f"predictions_{algorithm_pack[i][2]}_{current_time}.csv"
    output_df.to_csv(output_csv_name, index=False)
    
    # Algoritma untuk output data yang dipakai untuk melatih model
    clean_data_df = pd.DataFrame(x_train)
    clean_data_df.columns = all_column_names
    clean_data_csv_name = f"clean_data_{algorithm_pack[i][2]}_{current_time}.csv"
    clean_data_df.to_csv(clean_data_csv_name, index=False)

    # Algoritma untuk output dokumentasi seperti permasalahan yang diselesaikan, waktu run code, dll.
    documentation_txt_name = f"documentation_{algorithm_pack[i][2]}_{current_time}.txt"
    current_ipynb_file = "PHOoH.ipynb"
    string_to_write = ""
    with open(documentation_txt_name, "w") as f:
        # Write your documentation content to the file
        string_to_write += f"Problem to solve\t\t: {os.path.basename(os.getcwd())}\n"
        string_to_write += f"Date and Time file created\t: {get_current_time()}\n"
        string_to_write += f"Algorithm title\t\t\t: {algorithm_pack[i][1]}\n\n"
        string_to_write += f"Training data info:\n{train_data.dtypes}\n\n"
        string_to_write += f"Test data info:\n{test_data.dtypes}\n\n"
        string_to_write += f"Categorical Features\t: {categorical_features}\n"
        string_to_write += f"Numerical Features\t: {numerical_features}\n\n"
        string_to_write += f"For more data details see {clean_data_csv_name} (data that are beeing trained)\n"
        string_to_write += f"For data preprocessing details see {current_ipynb_file} or self\n"
        f.write(string_to_write)

    with zipfile.ZipFile(f"complete_data_{algorithm_pack[i][2]}_{current_time}.zip", "w") as zip_file:
        # Add the current code into the zip
        zip_file.write(current_ipynb_file, arcname=current_ipynb_file)
        # Add the CSV file
        zip_file.write(output_csv_name, arcname=output_csv_name)  # Specify archive name for the CSV
        zip_file.write(clean_data_csv_name, arcname=clean_data_csv_name)  # Specify archive name for the CSV
        # Add the text file
        zip_file.write(documentation_txt_name, arcname=documentation_txt_name)  # Specify archive name for the text file
        remove_file(clean_data_csv_name)
        remove_file(documentation_txt_name)

# -- Fungsi untuk output semua model yang dipakai

In [ ]:
def output_all_csv():
    for i in range(len(algorithm_pack)):
        output_one_csv(i)

def output_mandatory_csv():
     for i in range(3):
        output_one_csv(i)

# -- Memudahkan untuk outputnya

In [ ]:
choose = get_integer_option(0, 2, "What do you want to do with the csv?\n0. Train the csv data\n1. Get the info of the csv data\n2. Get the csv of clean preprocessed data\n[0/1/2]: ")

if(choose == 0):
    choose = get_integer_option(0, 2, "Use all model?\n0. No\n1. Yes\n2. Mandatory\n[0/1/2]: ")
    if(choose == 0):
        choose = get_integer_option(0, len(algorithm_pack) - 1, f"Which model to use?"+algorithm_option_string())
        output_one_csv(choose)
    elif(choose == 1):
        output_all_csv()
    elif(choose == 2):
        output_mandatory_csv()
elif(choose == 1):
    print("Training data info:\n",train_data.dtypes,"\n")
    print("Test data info:\n",test_data.dtypes)
    print("Categorical Features: ",categorical_features)
    print("Numerical Features: ",numerical_features)
elif(choose == 2):
    clean_data_df = pd.DataFrame(x_train)
    clean_data_df.columns = all_column_names
    clean_data_df.to_csv('clean_data.csv', index=False)

In [ ]:
# Run All